In [ ]:
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from PIL import Image, ImageOps
from plotly.subplots import make_subplots

from os import getcwd

%matplotlib inline


In [ ]:
def normalize(arr):
    min_val = abs(np.min(arr))
    max_val = abs(np.max(arr))
    return (arr + min_val) / (min_val + max_val)


def img_normalize(img):
    img_max = np.max(img)
    return 255 * img / img_max


def find_ft(img):
    ft = np.fft.fft2(img)
    return np.fft.fftshift(ft)


def find_ift(ft):
    ift = np.fft.ifftshift(ft)
    return np.fft.ifft2(ift)


def pink_filter_mask(x_freq, y_freq, factor=2.4):
    eps = 10 ** -8
    x, y = np.meshgrid(x_freq, y_freq)
    f = np.hypot(x, y)
    f = f ** factor + eps
    return normalize(1 / f)


def sharp_round_filter(x_freq, y_freq, radius, reverse=False):
    eps = 10 ** -8
    x, y = np.meshgrid(x_freq, y_freq)
    if reverse:
        f = np.zeros((x_size, y_size))
    else:
        f = np.ones((x_size, y_size))
    
    for i, xx in enumerate(x_freq):
        for j, yy in enumerate(y_freq):
            if xx ** 2 + yy ** 2 <= radius ** 2:
                if reverse:
                    f[i, j] = 1
                else:
                    f[i, j] = 0
    return normalize(f)


def sharp_square_filter(x_freq, y_freq, width, reverse=False):
    eps = 10 ** -8
    x, y = np.meshgrid(x_freq, y_freq)
    if reverse:
        f = np.zeros((x_size, y_size))
    else:
        f = np.ones((x_size, y_size))
    
    for i, xx in enumerate(x_freq):
        for j, yy in enumerate(y_freq):
            if abs(xx) + abs(yy) <= width:
                if reverse:
                    f[i, j] = 1
                else:
                    f[i, j] = 0
    return normalize(f)


CURRENT_DIR = getcwd()


# Rubik's cube example

In [ ]:
cube = Image.open(CURRENT_DIR + '/img/cube_1.png').convert('L')
cube_fr = find_ft(cube)

x_size, y_size = cube_fr.shape
xx = np.linspace(-x_size / 2, x_size / 2, x_size)
yy = np.linspace(-y_size / 2, y_size / 2, y_size)

kernel = normalize(sharp_square_filter(yy, xx, width=100))
eps = 10 ** -10

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10, 10))
ax1.imshow(cube, cmap='gray', vmin=0, vmax=255)
# Wrong scaling, but it gives a really interesting visual representation
ax2.imshow(find_ift(cube_fr * kernel).real, cmap='gray', vmin=0, vmax=np.max(np.log(np.absolute(cube_fr + eps))))


In [ ]:
cube = Image.open(CURRENT_DIR + '/img/woman.jpeg').convert('L')
cube_fr = find_ft(cube)

x_size, y_size = cube_fr.shape
xx = np.linspace(-x_size / 2, x_size / 2, x_size)
yy = np.linspace(-y_size / 2, y_size / 2, y_size)

fps = 30
seconds_num = 5
x_mesh, y_mesh = np.meshgrid(xx, yy)

radius = 300
cube_freq = [cube]
kernel_arr = []

for i in range(fps * seconds_num):
    kernel = normalize(sharp_square_filter(xx, yy, width=i / fps / seconds_num * 150))
    kernel_arr.append(kernel)
    cube_freq.append(img_normalize(find_ift(cube_fr * kernel).real))
    

In [ ]:
%matplotlib notebook

fig = plt.figure(figsize=(8,8))
im = plt.imshow(cube_freq[0], interpolation='none', aspect='equal', cmap='gray', vmin=0, vmax=255)


def animate_func(i): 
    im.set_array(cube_freq[i])
    return [im]


anim = animation.FuncAnimation(
    fig, 
    animate_func, 
    frames = seconds_num * fps,
    interval = 1000 / fps, # in ms
)


# anim.save('women.gif', writer='pillow')


In [ ]:
%matplotlib inline

cube = Image.open(CURRENT_DIR + '/img/cube_1.png').convert('L')
cube_fr = find_ft(cube)

eps = 10 ** -10

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(10, 10))
ax1.imshow(cube, cmap='gray', vmin=0, vmax=255)
ax2.imshow(np.log(np.absolute(cube_fr + eps)), cmap='gray')


# Phase correlation example

In [ ]:
f1 = Image.open(CURRENT_DIR + '/img/horse_1.png').convert('L')
f2 = Image.open(CURRENT_DIR + '/img/horse_translated_1.png').convert('L')

f1_freq = find_ft(f1)
f2_freq = find_ft(f2)

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 15))
ax1.imshow(f1, cmap='gray', vmin=0, vmax=255)
ax2.imshow(f2, cmap='gray', vmin=0, vmax=255)


In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 15))
ax1.imshow(np.log(np.absolute(f1_freq)), cmap='gray')
ax2.imshow(np.log(np.absolute(f2_freq)), cmap='gray')


In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 15))
ax1.imshow(np.angle(f1_freq, deg=True), cmap='gray')
ax2.imshow(np.angle(f2_freq, deg=True), cmap='gray')


In [ ]:
ncps = f1_freq * np.conj(f2_freq) / np.abs(f1_freq * f2_freq)
shift = find_ift(ncps)

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 15))
ax1.imshow(np.absolute(ncps), cmap='gray')
ax2.imshow(np.absolute(shift), cmap='gray')


In [ ]:
ind = np.unravel_index(np.argmax(shift, axis=None), shift.shape)
[i - j for i, j in zip(shift.shape, ind)]


# Shifting Pink Noise 

In [ ]:
x_size = 256
y_size = 256

xx = np.linspace(-x_size / 2, x_size / 2, x_size)
yy = np.linspace(-y_size / 2, y_size / 2, y_size)

# Generating cloud image
whitenoise = np.random.normal(0, 1, (y_size, x_size))
ft_arr = find_ft(whitenoise)
kernel = pink_filter_mask(xx, yy, factor=2.4)

pink_ft_arr = ft_arr * kernel
pink_noise = find_ift(pink_ft_arr).real

# Shifting initial image
dx = 25
dy = 20

x_mesh, y_mesh = np.meshgrid(xx, yy)
shift = np.exp(-1j * 2 * np.pi  * (x_mesh * dx / y_size + y_mesh * dy / x_size));

# Apply the phase shift along both axes
f_freq_shifted = pink_ft_arr * shift
shifted_pink_noise = find_ift(f_freq_shifted)

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(15, 15))
ax1.imshow(pink_noise.real, cmap='gray')
ax2.imshow(shifted_pink_noise.real, cmap='gray')


In [ ]:
%matplotlib notebook

# Generating cloud image
whitenoise = np.random.normal(0, 1, (y_size, x_size))
ft_arr = find_ft(whitenoise)
kernel = pink_filter_mask(xx, yy, factor=2)

pink_ft_arr = ft_arr * kernel
pink_noise = find_ift(pink_ft_arr).real

# Shift values
dx = 25
dy = 50

# Shift values per one frame
fps = 30
seconds_num = 5
dxx = dx / fps / seconds_num
dyy = dy / fps / seconds_num

x_mesh, y_mesh = np.meshgrid(xx, yy)
shift = np.exp(-1j * 2 * np.pi  * (x_mesh * dxx / y_size + y_mesh * dyy / x_size));

snapshots_freq = [pink_ft_arr]
snapshots_spatial = [pink_noise]
for _ in range(fps * seconds_num):
    # print(np.sum(np.absolute(snapshots_freq[-1]) ** 2))
    snapshots_freq.append(snapshots_freq[-1] * shift)
    snapshots_spatial.append(find_ift(snapshots_freq[-1]).real)

    
fig = plt.figure(figsize=(8,8))
frame = snapshots_spatial[0]
im = plt.imshow(frame, interpolation='none', aspect='auto', cmap='gray')


def animate_func(i):
    im.set_array(snapshots_spatial[i])
    plt.title(f"X shift {i * dxx:.2f} \n Y shift {i * dyy:.2f}")
    return [im]


anim = animation.FuncAnimation(
    fig, 
    animate_func, 
    frames = seconds_num * fps,
    interval = 1000 / fps, # in ms
)
